In [4]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="pchlenski/gpt2-transcoders", allow_patterns=["*.pt"],
    local_dir="/workspace/transcoder_circuits/gpt-2-small-transcoders", local_dir_use_symlinks=False
)

Fetching 26 files: 100%|██████████| 26/26 [00:04<00:00,  5.82it/s]


'/workspace/transcoder_circuits/gpt-2-small-transcoders'

In [5]:
# flake8: noqa
from transcoder_circuits.circuit_analysis import *
from transcoder_circuits.feature_dashboards import *
from transcoder_circuits.replacement_ctx import *
from sae_training.sparse_autoencoder import SparseAutoencoder

import transformer_lens as tl

model = tl.HookedTransformer.from_pretrained("gpt2")
transcoder_template = "/workspace/transcoder_circuits/gpt-2-small-transcoders/final_sparse_autoencoder_gpt2-small_blocks.{}.ln2.hook_normalized_24576"
transcoders = []
for i in range(model.cfg.n_layers):
    transcoders.append(SparseAutoencoder.load_from_pretrained(f"{transcoder_template.format(i)}.pt").eval())

/workspace/transcoder_circuits/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2 into HookedTransformer


In [14]:
1+1

2

In [15]:
hook_point: tl.hook_points.HookPoint
for hook_point in model.hook_points():
    print(hook_point.name)

hook_embed
hook_pos_embed
blocks.0.ln1.hook_scale
blocks.0.ln1.hook_normalized
blocks.0.ln2.hook_scale
blocks.0.ln2.hook_normalized
blocks.0.attn.hook_k
blocks.0.attn.hook_q
blocks.0.attn.hook_v
blocks.0.attn.hook_z
blocks.0.attn.hook_attn_scores
blocks.0.attn.hook_pattern
blocks.0.attn.hook_result
blocks.0.mlp.hook_pre
blocks.0.mlp.hook_post
blocks.0.hook_attn_in
blocks.0.hook_q_input
blocks.0.hook_k_input
blocks.0.hook_v_input
blocks.0.hook_mlp_in
blocks.0.hook_attn_out
blocks.0.hook_mlp_out
blocks.0.hook_resid_pre
blocks.0.hook_resid_mid
blocks.0.hook_resid_post
blocks.1.ln1.hook_scale
blocks.1.ln1.hook_normalized
blocks.1.ln2.hook_scale
blocks.1.ln2.hook_normalized
blocks.1.attn.hook_k
blocks.1.attn.hook_q
blocks.1.attn.hook_v
blocks.1.attn.hook_z
blocks.1.attn.hook_attn_scores
blocks.1.attn.hook_pattern
blocks.1.attn.hook_result
blocks.1.mlp.hook_pre
blocks.1.mlp.hook_post
blocks.1.hook_attn_in
blocks.1.hook_q_input
blocks.1.hook_k_input
blocks.1.hook_v_input
blocks.1.hook_mlp_in


In [ ]:
# 
text = "When John and Mary went to the shops, John gave the bag to Mary"

def metric(model, text): 
    loss, _ = model.run_with_cache(text, loss_per_token=True)
    return loss[0,-1]

In [ ]:
activations = {}

def get_layer(name):
    return int(name.split(".")[1])

def cache_mlp_in_hook(acts, hook_point):
    assert "mlp.hook_pre" in hook_point.name
    global activations
    activations[hook_point.name] = acts

def patch_mlp_out_hook(acts, hook_point):
    assert "mlp.hook_post" in hook_point.name
    global activations
    inputs = activations[hook_point.name.replace("mlp.hook_post", "mlp.hook_pre")]
    transcoder = transcoders[get_layer(hook_point.name)]
    
    transcoder_out = transcoder(inputs)[0]
    transcoder_err = (acts - transcoder_out.detach())
    acts_patch = transcoder_err + transcoder_out
    return acts_patch

In [ ]:
for layer in range(model.cfg.n_layers):
    
    model.add_hook(f"blocks.{layer}.hook_mlp_in", cache_mlp_in_hook, "fwd")